# INF367 Mandatory 2

In [66]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from qiskit.circuit.library import UnitaryGate
from collections import Counter, defaultdict
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer import AerSimulator, QasmSimulator
from qiskit_ibm_runtime import SamplerV2 as Sampler
from matplotlib import pyplot as plt
from qiskit.result import marginal_counts
from src import Model1, Model2


In [67]:
SEED = 367

## Data exploration and pre-processing

In [ ]:
X, y = load_iris(return_X_y=True)
X_train, X_rest, y_train, y_rest = train_test_split(X, y, train_size=.6, random_state=SEED)
X_val, X_test, y_val, y_test = train_test_split(X_rest, y_rest, train_size=.5, random_state=SEED)
print("Training size: ", len(X_train))
print("Validation size: ", len(X_val))
print("Test size: ", len(X_test))

In [ ]:
print("Features shape: ",X_train.shape)
print("Target shape: ",y_train.shape)
print(f"Feature value range: {np.min(X_train)} : {np.max(X_train)}")
print("Target values: ", Counter(y_train))

In [ ]:
scaler = MinMaxScaler(feature_range=(0,np.pi))
X_train = scaler.fit_transform(X_train)

### Helper functions

In [69]:
def output_mapping(binary_counts, n_classes):
    """
    Maps binary count output to n classes
    """
    mapping = {}
    class_preds = defaultdict(int)
    for binary in binary_counts:
        b_number = int(binary, base=2)
        c = b_number % n_classes
        mapping.update({binary:c})
        
    for binary, count in binary_counts.items():
        class_preds[mapping[binary]] += count

    for i in range(n_classes):
        if i not in class_preds:
            class_preds[i] = 0
    
    return class_preds

In [70]:
BACKEND = AerSimulator()
def measure_circuit(qc: QuantumCircuit, shots:int=1000, qbits:list=None):
    pm = generate_preset_pass_manager(backend=BACKEND, optimization_level=1)
    isa_circuit = pm.run(qc)
    result = BACKEND.run(isa_circuit, shots=shots).result()
    if qbits is not None:
        return marginal_counts(result, qbits).get_counts()
    else:
        return result.get_counts()

## QNN-circuits

In [ ]:
# Qircuit 1
def make_circuit1(features, parameters):
    qc_1 = QuantumCircuit(4, 4)
    for i in range(4):
        qc_1.rx(qubit=i, theta=features[i])
    qc_1.barrier()

    #qc_1_unitary_V2 = UnitaryGate([[1,0], [0,1]])
    params = [Parameter(f"{i}") for i in range(9)]
    qc_1.append(Custom_UnitaryGate1([params[0], params[1]]), [3,2])
    qc_1.append(Custom_UnitaryGate1([params[0], params[1]]), [1,0])
    qc_1.barrier()

    qc_1.measure(qubit=2, cbit=2)
    qc_1.append(Custom_UnitaryGateV([params[2], params[3], params[4]]).control(1), [2,3])

    qc_1.measure(qubit=0, cbit=0)
    qc_1.append(Custom_UnitaryGateV([params[2], params[3], params[4]]).control(1), [0,1])
    
    qc_1.barrier()
    qc_1.append(Custom_UnitaryGate2([params[5], params[6], params[7], params[8]]), [3,1])
    qc_1.measure(qubit=1, cbit=1)
    qc_1.measure(qubit=3, cbit=3)

    qc_1 = qc_1.assign_parameters(parameters)
    return qc_1

def Custom_UnitaryGate1(parameters):
    qc = QuantumCircuit(2)
    qc.cx(0,1)
    qc.rz(parameters[0], 1)
    qc.ry(parameters[1], 0)
    qc.cx(0,1)
    return qc.to_gate(label="U1")

def Custom_UnitaryGate2(parameters):
    qc = QuantumCircuit(2)
    qc.cx(0,1)
    qc.rx(parameters[0], 1)
    qc.rx(parameters[1], 0)
    qc.rz(parameters[2], 0)
    qc.rz(parameters[3], 1)
    qc.cx(0,1)
    return qc.to_gate(label="U2")

def Custom_UnitaryGateV(parameters):
    qc = QuantumCircuit(1)
    qc.rx(parameters[0], 0)
    qc.ry(parameters[1], 0)
    qc.rz(parameters[2], 0)
    return qc.to_gate(label="V")

circ1 = make_circuit1([1.61567622, 1.83259571, 2.34288266, 3.00500167], [1,2,1,2,1,2,1,2,3])
circ1.decompose()
circ1.draw(reverse_bits=True, output="mpl")

In [ ]:
results = measure_circuit(circ1, 1000)
plot_histogram(results)
# output_mapping(results, 3)

In [ ]:
# Circuit 2

def circuit2(features, trainable_parameters, layers=2):
    input_size = len(features)
    qc = QuantumCircuit(input_size)
    for i in range(input_size):
        qc.rx(features[i], i) # Roterer den aktuelle qubit med en rx-rotasjon basert på input-verdien. Dette er input-encoding-steget, hvor vi mapper de klassiske inputverdiene til kvantetilstander.
    qc.barrier() # Vi legger inn en barrier for å markere slutten av input-encodingen. Her legger vi bare en sluttpunkt av circuit, den stripete linjen p circuits nede i ploten. 

    for i in range(layers): # Så går vi gjennom layers antall lag:
        for j in range(input_size):
            qc.ry(Parameter(f"phi{i}{j}"),j) # Roterer hver qubit med en ry-rotasjon, hvor rotasjonsvinkelen er en justerbar parameter. Vekslende ry-rotasjoner og CX-porter gir oss en justerbar krets.
        for j in range(input_size-1, 0, -1): 
            qc.cx(j, j-1) # Her legger vi qubitsene i entangelment tilstand
        qc.barrier() # Markerer slutten av laget. 

    qc = qc.assign_parameters(trainable_parameters) # Etter lagene binder vi de justerbare parameterne til kretsen ved hjelp av assign_parameters().
    qc.measure_all() # Til slutt måler vi alle qubitene ved hjelp av measure_all().
    return qc
    
layers = 2
feature_size = 4
parameters = np.random.uniform(low=0, high=np.pi, size=(layers*feature_size,))
circ2 = circuit2([0, 1.64, 2.24, 3.0], parameters, layers)
circ2.draw("mpl", reverse_bits=True)

In [ ]:
def circuit3(features, trainable_parameters, layers):
    input_size = len(features)
    qc = QuantumCircuit(input_size)

    # Step 1: Feature Encoding
    for i in range(input_size):
        qc.h(i)  # Legger til en Hadamard-port på hver qubit, som gir en superposisjonstilstand.
        qc.rz(features[i], i)  # RZ-porten er en av flere mulige porter vi kan bruke for encoding. Merk, kan bruke (RX, RY eller RZ). 
    qc.barrier()

    # Step 2: Variational Layers
    for i in range(layers):
        for j in range(input_size):
            qc.rx(Parameter(f"theta{i}{j}"), j)  # Justerbar RX-rotasjon på hver qubit.
            qc.ry(Parameter(f"phi{i}{j}"), j)    # Legger til en justerbar RY-rotasjon.
        for j in range(0, input_size - 1, 2):  # For å skape entanglement, legger vi til CX-porter mellom hvert par av qubits.
            qc.cx(j, j + 1)
        qc.barrier()

    # Step 3: Parameter Binding and Measurement
    qc = qc.assign_parameters(trainable_parameters)
    qc.measure_all()

    return qc

# Konfigurer parametre na 
layers = 4
feature_size = 3
parameters = np.random.uniform(low=0, high=2*np.pi, size=(layers * feature_size))
circ = circuit2([1.2, 2.1, 0.5], parameters, layers)
circ.draw("mpl", reverse_bits=True)

In [ ]:
results = measure_circuit(circ2, 500)
plot_histogram(results)
test = output_mapping(results, 3)
print(test)
print(max(test, key=lambda x: test[x]))

In [ ]:
# Circuit 3

qc_3 = QuantumCircuit()

## Gradient Descent Function

In [ ]:
class TestModel():
    def __init__(self, circuit_func, layers=2, learning_rate=0.01, prediction_shots=1000, gradient_shots=100, epsilon=1):
        self.layers = layers
        self.feature_size = 4
        self.parameters = np.random.uniform(low=0, high=np.pi, size=(layers*feature_size,))
        self.circuit_func = circuit_func
        self.learning_rate = learning_rate
        self.gradient_shots = gradient_shots
        self.prediction_shots = prediction_shots
        self.train_loss = []
        self.val_loss = []
        self.epsilon = epsilon

    def gradient(self, data, targets):
        gradients = []
        for i in range(len(self.parameters)):
            p1 = self.parameters.copy()
            p1[i] += self.epsilon
            p2 = self.parameters.copy()
            p2[i] -= self.epsilon
            loss1 = self._loss(data, targets, self.gradient_shots, p1)
            loss2 = self._loss(data, targets, self.gradient_shots, p2)
            partial = (loss1 - loss2)/2*self.epsilon
            gradients.append(partial)
        return np.array(gradients)
        
    
    def _loss(self, data, targets, shots, parameters=None):
        if parameters is None:
            parameters = self.parameters
        preds = np.array([self._predict_proba(x, shots=shots, parameters=parameters) for x in data])
        return log_loss(targets, preds, labels=[0,1,2])
        

    def fit(self, data, targets, val_data=None, val_targets=None):
        for i in range(5): 
            print(f"Epoch {i+1}", end=" ")
            gradient = self.gradient(data, targets)
            self.parameters -= self.learning_rate * gradient

            train_loss = self._loss(data, targets, shots=self.prediction_shots)
            print(f"Train loss: {train_loss}", end=" ")
            self.train_loss.append(train_loss)
            if val_data is not None and val_targets is not None:
                val_loss = self._loss(val_data, val_targets, shots=self.prediction_shots)
                self.val_loss.append(val_loss)
                print(f"Validation loss: {val_loss}")
        return self

    def _predict(self, x, shots=1000):
        circuit = self.circuit_func(x, self.parameters, self.layers)
        results = measure_circuit(circuit, shots)
        class_output = output_mapping(results, 3)
        prediction = max(class_output, key=lambda x: class_output[x])
        return prediction

    def predict(self, X):
        return np.array([self._predict(x, shots=self.prediction_shots) for x in X])
            

    def _predict_proba(self, x, shots=100, parameters=None):
        if parameters is None:
            parameters = self.parameters
            
        circuit = self.circuit_func(x, parameters, self.layers)
        results = measure_circuit(circuit, shots)
        class_output = sorted(output_mapping(results, 3).items())
        probs = np.array([n/shots for _,n in class_output])
        return probs
        
    def predict_proba(self, X):
        return np.array([self._predict_proba(x, shots=self.prediction_shots) for x in X])

## Training and Validation

In [ ]:
# test_model = TestModel(circuit2, layers=2, learning_rate=0.6, epsilon=1, gradient_shots=10)
test_model = Model2(layers=2, learning_rate=0.6, epsilon=1, gradient_shots=1000)
      
trained_model = test_model.fit(5, X_train, y_train, X_val, y_val)

In [72]:
def plot_loss(train_loss, val_loss):
    plt.title("Training and Validation loss")
    plt.plot(train_loss, label="Training")
    plt.plot(val_loss, label="Validation")
    plt.legend()
    plt.show()

In [ ]:
plot_loss(trained_model.train_loss, trained_model.val_loss)

## Test Performance

In [74]:
preds = trained_model.predict(X_test)

In [ ]:
print(accuracy_score(y_test, preds))

In [ ]:
cm = confusion_matrix(y_test, preds)
disp = ConfusionMatrixDisplay(cm)
disp.plot()
plt.show()